# Intro to machine learning

## Prerequisites
- [Basic Python](https://github.com/AI-Core/Python/blob/master/Intro%20to%20Python%20solutions.ipynb)

## Why do we care about machine learning?

If we had a model of the world (like a perfect simulation) then we'd be able to solve all problems.
We'd be able to simulate what the weather would be like tomorrow. 
Or what drugs will be effective for treating diseases.
Or what combination of words will make someone do exactly what you want.
Or all other problems.

**Building machines which can model the world (make accurate predictions about it) is the goal of AI.**

However, we actually never have a perfect model of the world - the world is too complex!

"All models are wrong, but some are useful" - George Box

Because we can't have perfect models, we have to settle with predictive models. 

In order to achieve goals, animals have to understand how the world works. 
Over time they build up mental models of how the world behaves. 
If I see clouds I can predict how likely it is to rain.
If I push this chair i can predict whether it will slide across the floor or topple over.
For these simple examples, we can make pretty good predictions.
This is probably because we have experienced a lot of similar situations, and can generalise

Machine learning is all about learning to represent the relationships between inputs and outputs. These inputs and outputs can take many forms.

![](images/inp-out.jpg)

We'd like to be able to build algorithms that can learn to use inputs to predict useful outputs.

Almost all machine learning algorithms consist of 4 components:
1. the data
2. the model
3. the criterion
4. the optimiser

This notebook will introduce you to all of those, with simple, practical examples.

## Side note - three types of Machine Learning

There are three main categories of problems within Machine Learning. It's worth knowing these straight away.

### **Unsupervised learning** 
Where we only have an input and try to predict something useful as an output, without being explicitly shown examples of what the output should be. This  what data is likely to in order to better understand the underlying structure of it. E.g. we have data about houses and try to split these examples into different clusters.

### **Reinforcement Learning**
Where our algorithm interacts with it's evironment and has to learn what actions to take to perform a task. E.g. we are trying to get an robot to walk or an algorithm to learn how to win at chess.

### **Supervised Learning**
Where we predict an output from a input, given examples of input-output pairs. E.g. we use different features about a house such as location, number of rooms, etc and try to predict the price.

Common synonyms
- Loss funtion = cost function = criterion = error function
- Inputs = Features
- Outputs = Labels

In this notebook, and the first series of lectures, we will be learning about supervised learning - where we have datasets with both input features and output labels. Later notebooks will cover unsupervised learning and reinforcement learning. 

## 1. The data - What problem are we solving?
Lets create a function that generates some artificial data.

The function should return an array of features and an array of labels which have a linear relationship (straight line, i.e. $y=wx + b$).
Both the features and labels should have length $m$ - which is an argument to the function. 
Although data collected in the real world often has much more complex correlations, linear functions make simple  test cases to learn about machine learning algorithms.

![image](images/data.jpg)
![image](images/labels.jpg)


As shown above, our supervised dataset consists of $m$ inputs. Each of these inputs has $n$ features (n-dimensional feature vectors). And each input has a corresponding label (it's a supervised dataset).

As we illustrated earlier, inputs and outputs can take many forms. The inputs do not have to be vectors and the labels do not have to be scalars as shown in the example.

- Image inputs could be matrices with width and height, rather than just vector length.
- Inputs could only have one feature, in which case they would be scalars (this is what we will implement now)

- If we were trying to classify an input as a member of a particular, discrete class (classifying dogs vs cats vs turtles for example), then we would have a vector output with the length of the number of classes - where each of those elements represents the confidence with which our model predicts the input to be a member of that corresponding class.
- Outputs could be an image or a video or a sound wave or spectrogram


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sample_linear_data(m=20): 
    ground_truth_w = 2.3 # slope (weight)
    ground_truth_b = -8 # intercept (bias)
    X =  # m random values from a random normal distribution
    Y =  # compute the output (with some random noise added)
    return X, Y # returns X (the input) and Y (labels)

def plot_data(X, Y):
    plt.figure() # create a figure
    plt.scatter(X, Y, c='r') # plot the data in color=red
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()
    
m = 10
X, Y = sample_linear_data(m)
print('X:',X, '\n')
print('Y:',Y, '\n')
plot_data(X, Y)

## 2. The model - How can we make predictions?
We want to be able use information that we have, to predict useful information that we don't have. 
This is an input output problem - we have an input (info we know), we want an output (info we don't have).
Mathematical functions can represent input output relationships e.g. $y = 2x +3$.
So we will use mathematical functions to model the relationship between our inputs and outputs.
Our goal is to model the world, and use that to be able to intelligently infer a lot from a little.

Lets create a simple model that represents a straight line (linear) relationship between the input and output and use it to make predictions about outputs, given inputs.

Our model will be of the form $y = wx + b$.

![title](images/NN1_singlevar_lr_equation.JPG)

In [ ]:
class LinearHypothesis:
    def __init__(self): #initalize parameters 
        self.w =  # randomly initialise weight
        self.b =  # randomly initialise bias
        
    def __call__(self, X): # how do we calculate output from an input in our model?
        ypred = # make a prediction
        return ypred # return prediction
    
    def update_params(self, new_w, new_b):
        self.w = # set this instance's weights to the new weight value passed to the function
        self.b = # do the same for the bias

In [ ]:
H = # instantiate our linear model
y_hat = # make prediction
print('Input:',X, '\n')
print('W:', H.w, 'B:', H.b, '\n')
print('Prediction:', y_hat, '\n')

Lets visualise our hypothesis against the true features and labels

In [ ]:
def plot_h_vs_y(X, y_hat, Y):
    plt.figure()
    plt.scatter(X, Y, c='r', label='Label')
    plt.scatter(X, y_hat, c='b', label='Hypothesis', marker='x')
    plt.legend()
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()

In [ ]:
plot_h_vs_y(X, y_hat, Y)

## 3. The criterion - How do we know how good our model is?

#### The criterion will also be referred to as the loss function, cost function, error function.

Our criterion should be a measure of how bad our model is. We will use it to compare different models. As the model gets worse the loss function should return larger values.

### Mean squared error (MSE) loss

**One way** to evaluate the performance of a model that predicts continuous (not discrete or bounded) outputs is to use the mean squared error loss. This does exactly what you think: it calculates the error (difference between our model's prediction and the true label) and then squares it and takes the mean of those square errors for each example. Squaring any value makes it positive, so as long as the error is not zero it will increase the value of the loss - regardless of whether our prediction is below or above the value of the label, the values of that **squared** difference will increase the returned loss.

There are many other criterions that are useful for different tasks (e.g. cross entropy loss for classification)

Let's write a function to calculate the cost using the mean squared error loss function.

![title](images/NN1_cost_function.JPG)

In [ ]:
def L(y_hat, labels):
    errors = # calculate errors
    squared_errors = # square errors
    mean_squared_error = # calculate mean 
    return mean_squared_error # return loss

In [ ]:
cost = L(y_hat, Y)
print(cost)

## 4. The optimiser

The optimiser optimises our model. Most machine learning models are **parametric**, which means that the function which they represent depends on their parameters (in our case the weight (slope) and bias (intercept)). Different optimisers improve our models using different algorithms.

In this notebook we will implement some fundamental optimisation techniques: random search and grid search.

### Random Search
Random seach is the process of randomly choosing values within a specified range and testing them to evaluate how good they are. E.g. test random values between 0 and 10.

![](images/NN1_randomsearch.JPG)

In [ ]:
def random_search(n_samples, limit=7):
    best_weights = # no best weight found yet
    best_bias = # no best bias found yet
    lowest_cost = # initialize it very high
    for i in  # try this many different parameterisations
        w = np.random.uniform(-limit, limit) # randomly sample a weight within the limits of the search
        b = np.random.uniform(-limit, limit) # randomly sample a bias within the limits of the search
        H. # update our model with random parameters
        y_hat = # make prediction
        cost =  # calculate loss
        if  # if this is the best parameterisation so far
            lowest_cost =  # update the lowest running cost to the cost for this parameterisation
            best_weights = # get best weights so far from the model
            best_bias = # get best bias so far from the model
    print('Lowest cost of', lowest_cost, 'achieved with weight of', best_weights, 'and bias of', best_bias)
    return best_weights, best_bias

In [ ]:
best_weights, best_bias = random_search(100) # do 100 samples in a random search 
H. # make sure to set our model's weights to the best values we found
plot_h_vs_y(X, H(X), Y)

### What happened?

Our random search optimisation was able to fit the input-output relationship of our data! Or at least it got close. 

#### Why doesn't it get closer as we sample more potential parameterisations? 
This is because of the limits of the values of the parameters that we perform the grid search over. In this case, by default we are only trying parameters in the range from -7 to 7. But the true bias is -8 which is outside of this range. So we have made a mistake in assuming the range of values that our optimal parameterisation might be included in. Feel free to change this limit in the function definition to see the model converge. 

### Grid Search

Grid search is the process of trying out values at common intervals within a specified range and testing them to evaluate how good they are. E.g. test the values [0, 1, 2, 3, 4, 5]

![](images/NN1_gridsearch.JPG)

In [ ]:
from itertools import permutations
def generate_grid_search_values(n_params, n_samples=100, minval=-2.5, maxval=2.5):
    n_samples_per_param = int(np.power(n_samples, 1 / n_params)) # want 100 samples for 2 variables, so try 10 different values for each parameter because 10^2=100
    print(f'Trying {n_samples_per_param} samples per parameter')
    param_values =  # get list of different parameters to try
    grid_samples =  # try every possible permutation of the param values
    return grid_samples

def grid_search(grid_search_values):
    best_weights =  # no best weight found yet
    best_bias =  # no best bias found yet
    lowest_cost =  # initialize it very high
    for search_val in grid_search_values: # for each model parameterisation that we will try 
        # update model parameters
        y_hat = # make prediction
        cost = # calculate loss
        if  # if this is the best parameterisation so far
            lowest_cost =  # update the lowest running cost to the cost for this parameterisation
            best_weights =  # get best weights so far from the model
            best_bias =  # get best bias so far from the model
    print('Lowest cost of', lowest_cost, 'achieved with weight of', best_weights, 'and bias of', best_bias)
    return best_weights, best_bias

In [ ]:

# try using 100 grid search values
grid_search_values = generate_grid_search_values(2, n_samples=100) # generate model parameterisations to try
best_weights, best_bias = grid_search(grid_search_values) # perform grid search
H. # update model with best parameters found
plot_h_vs_y(X, H(X), Y) # plot predictions and true labels

grid_search_values = generate_grid_search_values(2, n_samples=99) # generate model parameterisations to try
best_weights, best_bias = grid_search(grid_search_values) # perform grid search
H.update_params(best_weights, best_bias) # update model with best parameters found
plot_h_vs_y(X, H(X), Y) # plot predictions and true labels

### What happened?

#### Why the difference in predictions from 100 samples (pretty good) and 99 samples (pretty bad)? 

Well, grid search will only test parameterisations that are exactly on it's grid. In the case of the 99 samples, the optimal parameters do exist within this region, but they dont lie exactly on the grid - they lie between points that we test on the grid. Hence those optimal parameters are not found. 

With the 100 samples, one of the parameterisations (pairs of weight and bias) on our grid lies close to the actual optimal values. So the model manages to much better represent the input-output relationship between our features and labels.

Another danger would be that the space which we are searching with our grid does not contain the optimal parameterisation.

### Will random search and grid search get us all the way?

Aside from the issues showcased above, the major limitation of these search methods is how they scale with the number of parameters in our model. To model more complex functions we'll need more complex models - models with more parameters. But the time taken for these search methods scales **exponentially** with the number of parameters. This is because these methods have to search the whole space, and they keep searching even if they find the optimal value (they can't be sure it's the best parameterisation in the domain that they're checking until they've compared it to everywhere else.

## Yes, you made it!
In this notebook, we learnt the very basic recipe for making machine learning algorithms. This consisted of:
1. The data - our examples of inputs and outputs (in the supervised case) which determine the function that our model will learn to represent and hence the problem that we are solving
2. The model - our mathematical function that we pass our data forward through to make a prediction for the output
3. The criterion - how we measure how bad our model is. We used the mean squared error loss function.
4. The optimiser - our method for updating the parameters of our models. We tried out random search and grid search.


## Next steps
- [Gradient based optimisation](https://github.com/AI-Core/Strong-ML-Foundations/blob/master/Gradient%20based%20optimisation.ipynb) - in this notebook we will look at optimisation techniques that do scale to 
more complex models and problems